## **Analyze Fine-Tuning Performance on Individual Languages**

### **Objective**

This step evaluates the performance of the fine-tuned model on some individual languages (English, Spanish, and Chinese) to investigate whether the good overall performance is influenced by the imbalance in the dataset. By fine-tuning the model on each language separately, we can determine how well the model learns from datasets with varying sizes and linguistic properties

### **Step 1:  Import Required Libraries**

Load essential libraries for model training, evaluation, and data manipulation.

In [1]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

/home/yazata/venvs/ai_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-12 23:59:35.726890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736722775.751012  119332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736722775.758429  119332 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 23:59:35.784589: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical op

### **Step 2: Define Helper Functions**

Functions for tokenization, evaluation metrics, and processing datasets.

In [3]:
# Define metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).numpy()
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Define tokenization function
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )

### **Step 3: Load Data**

Load datasets for English, Spanish, and Chinese.

In [4]:
# File paths for language-specific datasets
language_files = {
    "English": "../data/language_datasets/English_data.csv",
    "Spanish": "../data/language_datasets/Spanish_data.csv",
    "Chinese": "../data/language_datasets/Chinese_data.csv",
}

# Dictionary to store datasets
datasets_by_language = {}

# Load datasets into Hugging Face format
for language, file_path in language_files.items():
    data = pd.read_csv(file_path)
    train_data, val_data = train_test_split(
        data, test_size=0.2, stratify=data["label"], random_state=123
    )
    train_dataset = Dataset.from_pandas(train_data)
    val_dataset = Dataset.from_pandas(val_data)
    datasets_by_language[language] = DatasetDict(
        {"train": train_dataset, "validation": val_dataset}
    )


### **Step 4: Define Training and Fine-Tuning Parameters**

Use the same pre-trained model (xlm-roberta-base) for consistency.

In [5]:
# Model name
model_name = "xlm-roberta-base"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir="./language_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./language_logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
)

/home/yazata/venvs/ai_env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### **Step 5: Fine-Tune Model for Each Language**

Loop through datasets for each language, fine-tune the model, and store results.

In [8]:
results = []  # Store results for each language

for language, dataset in datasets_by_language.items():
    print(f"\nFine-tuning model for language: {language}")
    try:
        # Tokenize dataset
        tokenized_dataset = dataset.map(
            lambda x: preprocess_function(x, tokenizer), batched=True
        )
        tokenized_dataset = tokenized_dataset.remove_columns(
            ["premise", "hypothesis", "lang_abv", "language"]
        )

        # Load model
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=3
        )
        model.to("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        # Train model
        #trainer.train()
        trainer.train(resume_from_checkpoint=True) # if the training is resumed


        # Evaluate and save results
        eval_results = trainer.evaluate()
        print(f"Validation Results for {language}: {eval_results}")
        results.append(
            {"language": language, "accuracy": eval_results["eval_accuracy"]}
        )

    except Exception as e:
        print(f"Error processing language {language}: {e}")


Fine-tuning model for language: English


Map: 100%|██████████| 1374/1374 [00:00<00:00, 2258.09 examples/s]
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_116067/3393261999.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/yazata/venvs/ai_env/lib/python3.12/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorc

{'loss': 0.1927, 'grad_norm': 57.126953125, 'learning_rate': 7.790697674418605e-06, 'epoch': 6.1}


 64%|██████▍   | 2200/3440 [01:11<02:55,  7.08it/s]  

{'loss': 0.1956, 'grad_norm': 48.05585479736328, 'learning_rate': 7.209302325581395e-06, 'epoch': 6.4}


...

Validation Results for Spanish: {'eval_loss': 0.8154340386390686, 'eval_accuracy': 0.6756756756756757, 'eval_f1': 0.6752668069113252, 'eval_runtime': 0.6202, 'eval_samples_per_second': 119.326, 'eval_steps_per_second': 8.063, 'epoch': 10.0}

Fine-tuning model for language: Chinese


Map: 100%|██████████| 83/83 [00:00<00:00, 7851.31 examples/s]
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_116067/3393261999.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/yazata/venvs/ai_env/lib/python3.12/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/bl

{'train_runtime': 0.2049, 'train_samples_per_second': 16005.106, 'train_steps_per_second': 1024.717, 'train_loss': 0.0, 'epoch': 10.0}


100%|██████████| 6/6 [00:00<00:00, 10.43it/s]

Validation Results for Chinese: {'eval_loss': 1.1679537296295166, 'eval_accuracy': 0.5903614457831325, 'eval_f1': 0.5713297231313469, 'eval_runtime': 0.7079, 'eval_samples_per_second': 117.251, 'eval_steps_per_second': 8.476, 'epoch': 10.0}


### **Step 6: Analyze Results**

Create a bar plot to compare validation accuracies across languages.

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.bar(results_df["language"], results_df["accuracy"], color="skyblue")
plt.title("Validation Accuracy by Language", fontsize=16)
plt.xlabel("Language", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("validation_accuracy_by_language.png")
plt.show()

# Save results to CSV
results_df.to_csv("language_fine_tuning_results.csv", index=False)
print("Results saved to language_fine_tuning_results.csv")

### **Step 7: Conclusion**

**The evaluation of the fine-tuned model on individual languages revealed the following key insights:**

-   **Performance on English:** The model achieved a validation accuracy of approximately **0.71 on English**, which is consistent with the performance observed for the entire dataset. This suggests that the model performs particularly well on English, likely due to its dominance in the training data.

-   **Performance on Spanish and Chinese:** For Spanish and Chinese, the validation accuracies were **0.67 and 0.59, respectively**. These lower results compared to English confirm that the model struggles with languages with fewer data points in the training set.

-   **Impact of Imbalanced Data:** The overall performance on the imbalanced dataset appears to be heavily influenced by English. Since English constitutes the majority of the training data, it likely biases the model to perform well on English while underperforming on other languages. The weighted average of individual performances, driven by the population size of each language, results in the observed overall accuracy.

-   Implications for Data Balancing: This analysis supports the hypothesis that **balancing the data could lead to better overall performance** by improving the model's ability to generalize across all languages, not just English. Balancing the training data would ensure equal representation of all languages, potentially addressing the observed bias.